In [76]:
import requests

In [78]:
API_KEY = "3xDBRMcbOCvZbZ6ydIRbN0wHoru7WrDLNyLVOMJX"
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_KEY}"

response = requests.get(url)

In [79]:
response = requests.get(url)

In [95]:
details = data['near_earth_objects']['2024-01-01'][0]

In [80]:
asteroids_data = []
target = 10000
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_KEY}"


while len(asteroids_data) < target:
    response = requests.get(url)
    data = response.json()
    details = data['near_earth_objects']
    for date,info in details.items():
      for i in info:
        asteroids_data.append(dict(id = i['id'],
        name = i['name'],
        mag= float(i['absolute_magnitude_h']),
        refid=i['neo_reference_id'],       
        estimated_diameter_min_km=i['estimated_diameter']['kilometers']['estimated_diameter_min'],
        estimated_diameter_max_km=i['estimated_diameter']['kilometers']['estimated_diameter_max'],
        is_potentially_hazardous_asteroid=bool(i['is_potentially_hazardous_asteroid']),
        relative_velocity_kmph=i['close_approach_data'][0]['relative_velocity']['kilometers_per_hour'],
        astronomical=i['close_approach_data'][0]['miss_distance']['astronomical'],
        miss_distance_km=i['close_approach_data'][0]['miss_distance']['kilometers'],
        miss_distance_lunar=i['close_approach_data'][0]['miss_distance']['lunar'],
        orbiting_body=i['close_approach_data'][0]['orbiting_body'],                      
        date =  i['close_approach_data'][0]['close_approach_date']))
        if len(asteroids_data) >= target:
            break
      if len(asteroids_data) >= target:
          break

url = data['links']['next']

In [28]:
asteroids_data

[{'id': '2415949',
  'name': '415949 (2001 XY10)',
  'mag': 19.37,
  'refid': '2415949',
  'estimated_diameter_min_km': 0.3552670883,
  'estimated_diameter_max_km': 0.7944013596,
  'is_potentially_hazardous_asteroid': False,
  'relative_velocity_kmph': '57205.8951204341',
  'astronomical': '0.3372535274',
  'miss_distance_km': '50452409.349026638',
  'miss_distance_lunar': '131.1916221586',
  'orbiting_body': 'Earth',
  'date': '2024-01-02'},
 {'id': '3160747',
  'name': '(2003 SR84)',
  'mag': 26.0,
  'refid': '3160747',
  'estimated_diameter_min_km': 0.0167708462,
  'estimated_diameter_max_km': 0.0375007522,
  'is_potentially_hazardous_asteroid': False,
  'relative_velocity_kmph': '38589.054833182',
  'astronomical': '0.1323425924',
  'miss_distance_km': '19798169.933318188',
  'miss_distance_lunar': '51.4812684436',
  'orbiting_body': 'Earth',
  'date': '2024-01-02'},
 {'id': '3309828',
  'name': '(2005 YQ96)',
  'mag': 20.62,
  'refid': '3309828',
  'estimated_diameter_min_km': 0.1

 database connection

In [1]:
!pip install mysql-connector-python

In [81]:
import mysql.connector as db

In [82]:
connection = db.connect(

                            host = "localhost", # IP address of your server
                            user = "sowmiya",
                            password = "tiger",
                            database = "nasa"
                    )

In [83]:
cursor = connection.cursor()

In [13]:
q1="create table nasa.asteroids(id int,name text,mag float,estimated_diameter_min_km float,estimated_diameter_max_km float,is_potentially_hazardous_asteroid bool)"

In [15]:
cursor.execute(q1)

In [74]:
insert_query = """
    INSERT INTO nasa.asteroids (
        id,
        name,
        mag,
        estimated_diameter_min_km,
        estimated_diameter_max_km,
        is_potentially_hazardous_asteroid
    ) VALUES (%s, %s, %s, %s, %s, %s)
"""

for i in asteroids_data:
    values = (
        i['id'],
        i['name'],
        i['mag'],
        i['estimated_diameter_min_km'],
        i['estimated_diameter_max_km'],
        i['is_potentially_hazardous_asteroid']
    )
    cursor.execute(insert_query, values)

connection.commit()


# Table close_approach

In [ ]:
create_query = """
CREATE TABLE close_approach (
    neo_reference_id INT,           
    close_approach_date DATE,        
    relative_velocity_kmph FLOAT,   
    astronomical FLOAT,              
    miss_distance_km FLOAT,         
    miss_distance_lunar FLOAT,      
    orbiting_body VARCHAR(255)     
)
"""

In [ ]:
cursor.execute(create_query)

In [92]:
insert_query = """
INSERT INTO close_approach (
    neo_reference_id, 
    close_approach_date, 
    relative_velocity_kmph, 
    astronomical, 
    miss_distance_km, 
    miss_distance_lunar, 
    orbiting_body
) 
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
for i in asteroids_data:
    data = (
        i['refid'],      # neo_reference_id
        i['date'],   # close_approach_date
        i['relative_velocity_kmph'],# relative_velocity_kmph
        i['astronomical'],          # astronomical (AU)
        i['miss_distance_km'],      # miss_distance_km
        i['miss_distance_lunar'],   # miss_distance_lunar (LD)
        i['orbiting_body']          # orbiting_body
    )
    
    # Execute the INSERT query for each record
    cursor.execute(insert_query, data)
connection.commit()